In [1]:
load_embeddings_from_file = False # True

In [2]:
from utils import html_tools
from dask.distributed import Client
import dask.bag as db

import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import faiss
import numpy as np

In [4]:
unique_drug_names = list(html_tools.get_unique_drugs())

/Users/ariellubonja/prog/drugs-embedding/utils/html_tools.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  drugs_df = drugs_df.applymap(replace_url_w_drug_name)


## Original pip install biobert-embedding does not work (it's too old)

I forked the repo. Install using `pip install git+https://github.com/ariellubonja/biobert_embedding`

### Word Embeddings of Drugs.com drug names (without supporting text)

Find the drug names I've crawled [here](https://drive.google.com/drive/folders/1EO659a-tyjfXjKzHk-M0WreZBR14MgRM?usp=sharing)

<font color="orange">The cell below will try to handle Out of Vocabulary terms. This means that the embedding matrix is not a clear matrix, but will be a list of Tensors</font>

In [5]:
if load_embeddings_from_file:
    biobert_embeddings_oov = torch.load('outputs/biobert_oov_embeddings.pt')
else:
    # If using Conda, you need to install this manually by cloning Ariel's repo, cd-ing to it, and running:
    #   /Users/ariellubonja/anaconda3/envs/faiss/bin/python (i.e. path to your Conda environment's python) setup.py install
    from biobert_embedding.embedding import BiobertEmbedding # Embeddings take much longer to run when this is installed from Conda
    
    biobert = BiobertEmbedding()
    # This will take a while
    client = Client()
    
    # db - Dask Bag
    b = db.from_sequence(unique_drug_names)

    # Map the function to the data
    biobert_embeddings_oov = b.map(biobert.word_vector).compute()

    torch.save(biobert_embeddings_oov, 'outputs/biobert_oov_embeddings.pt')


    # biobert_embeddings_oov = list(map(biobert.word_vector, unique_drug_names)) # Single-threaded
    # Out of vocabulary will be split into smaller. May end up with multiple embeddings for each word. Parallelized

    # Using Python's native multiprocessing library
    # biobert_embeddings_oov = pool.map(biobert.word_vector, unique_drug_names)

    # pool.close()
    # pool.join()

100%|█████████████████████████████| 436M/436M [00:11<00:00, 39.2MiB/s]


File Downloaded! It is stored in: models/pytorch_model.bin


100%|████████████████████████████████| 313/313 [00:00<00:00, 839kiB/s]


File Downloaded! It is stored in: models/config.json


100%|█████████████████████████████| 213k/213k [00:00<00:00, 3.73MiB/s]


File Downloaded! It is stored in: models/vocab.txt


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 414.03 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-02-26 17:05:35,578 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:53365'.
2024-02-26 17:05:35,579 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/distributed/comm/tcp.py", line 224, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/li

KeyboardInterrupt: 

2024-02-26 17:06:34,292 - distributed.nanny - WARNING - Restarting worker
2024-02-26 17:06:36,607 - distributed.nanny - WARNING - Restarting worker


In [ ]:
pd.DataFrame(biobert_embeddings_oov, index=unique_drug_names)

In [ ]:
# These are the indices of the drug names that have only one embedding, i.e. the drug name is in the Vocabulary
vector_embd_indices = [i for i, num in enumerate(biobert_embeddings_oov) if len(num) == 1]
len(vector_embd_indices)

In [ ]:
len(biobert_embeddings_oov) == len(unique_drug_names)

2024-02-26 17:06:32,876 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/distributed/nanny.py", line 976, in run
    await worker.finished()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/distributed/core.py", line 628, in finished
    await self._event_finished.wait()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/locks.py", line 213, in wait
    await fut
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", l

In [8]:
vector_embeddings = torch.stack([biobert_embeddings_oov[i][0] for i in vector_embd_indices]).numpy()
corresponding_drug_names = [unique_drug_names[i] for i in vector_embd_indices]

In [9]:
vector_embeddings.shape

(3310, 768)

In [10]:
drug_embeddings_df = pd.DataFrame(vector_embeddings, index=corresponding_drug_names)
drug_embeddings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
urecholine,11.469607,-0.776724,0.044609,-10.107757,6.564709,-3.624166,2.572928,5.349373,-0.149269,10.066398,...,-6.674558,-0.790161,-1.359481,12.615541,-0.082356,-4.356493,5.387687,-6.618952,-0.551896,1.513196
ceretec,-1.572915,-2.244111,-3.945309,-7.028200,0.050729,0.314049,2.837068,-1.046860,-1.842020,4.288139,...,-3.122034,2.518466,0.054589,-3.222387,0.699996,3.663987,2.502228,-1.997871,7.449434,-1.591506
romosozumab-aqqg-subcutaneous,6.957013,-19.181356,-3.446933,-4.090069,-4.467539,2.691726,4.566215,3.943487,6.578664,11.947012,...,-7.813425,-1.256110,-0.822931,5.539540,7.523422,2.037103,-0.225136,8.574185,-7.183357,-5.312540
velaglucerase-alfa-intravenous,2.807131,-2.837260,-0.728826,-3.728203,0.058904,-1.753445,4.136845,-0.992869,-0.514792,0.094263,...,-1.657840,1.985155,-3.756186,-1.528562,-1.311492,3.325174,5.887980,-6.094766,1.942933,1.433478
pramosone-e-cream,0.834622,-6.558954,-0.172969,-1.983035,-5.003409,-0.648337,3.053739,0.101664,2.051489,4.001922,...,-5.026463,-3.573544,4.945977,-2.239543,1.391859,1.739651,-0.483320,-0.966127,-0.914355,-2.714044


## Evaluations

Top-K similarities

In [18]:
help(round)

Help on built-in function round in module builtins:

round(number, ndigits=None)
    Round a number to a given precision in decimal digits.
    
    The return value is an integer if ndigits is omitted or None.  Otherwise
    the return value has the same type as the number.  ndigits may be negative.



In [19]:
round(0.04126324, 4)

0.0413

In [21]:
# Normalize the vectors to turn FAISS's Inner Product into Cosine Similarity
biobert_embeddings_np = drug_embeddings_df.values
biobert_embeddings_np = biobert_embeddings_np / np.linalg.norm(biobert_embeddings_np, axis=1)[:, None]


index = faiss.IndexFlatIP(drug_embeddings_df.values.shape[1])  # Inner-Product similarity of 768-dimensions


# Add biobert_embeddings_np to the index
index.add(biobert_embeddings_np)

# number of nearest neighbors
k = 5

# similarity search
# D contains the similarities to the nearest neighbors (cosine similarity)
# I contains the indices of the nearest neighbors (excluding self)
D, I = index.search(biobert_embeddings_np, k+1)  # Retrieve k+1 neighbors to exclude self


for i in range(biobert_embeddings_np.shape[0]): # Iterate over each embedding
    embedding = biobert_embeddings_np[i]
    nearest_indices = I[i][1:]  # Exclude self, start from index 1
    nearest_similarities = D[i][1:]

    # Print the top-k nearest neighbors for the current embedding
    print(f"Most similar (Nearest-neighbor) drugs to {drug_embeddings_df.index[i]}:")
    for j, index in enumerate(nearest_indices):
        similarity = nearest_similarities[j]
        print(f"{j+1}: {drug_embeddings_df.index[index]} , Similarity {round(similarity, 4)}")
    print()


Most similar (Nearest-neighbor) drugs to urecholine:
1: membraneblue )#, Similarity 0.9144999980926514
2: citranatal-harmony-3-0 )#, Similarity 0.9125000238418579
3: temsirolimus )#, Similarity 0.9039999842643738
4: isovue-m-300 )#, Similarity 0.8973000049591064
5: prevident-5000-dry-mouth )#, Similarity 0.8950999975204468

Most similar (Nearest-neighbor) drugs to ceretec:
1: genace )#, Similarity 0.9020000100135803
2: aricept )#, Similarity 0.8899999856948853
3: norpace-cr )#, Similarity 0.8845000267028809
4: operand-chg-surgical-scrub-brush )#, Similarity 0.8830000162124634
5: ony-clear )#, Similarity 0.8817999958992004

Most similar (Nearest-neighbor) drugs to romosozumab-aqqg-subcutaneous:
1: plan-b )#, Similarity 0.9017999768257141
2: vitamin-d )#, Similarity 0.8931999802589417
3: dextroamphetamine-saccharate-amphetamine-aspartate-dextroamphetamine-sulfate-amphetamine-sulfate )#, Similarity 0.883400022983551
4: neomycin-polymyxin-b-dexamethasone-ointment )#, Similarity 0.882099986

## 3194 of the Drugs have vector embeddings, whereas the rest return a Matrix that we have to deal with using Pooling

Let's see the 3194

<font color="red">Most of the drugs give a matrix as their embedding, and not a vector.</font>

Need to try various pooling methods to get a vector from the matrix + see what is best

## TODO try ClinicalBERT https://github.com/EmilyAlsentzer/clinicalBERT

In [4]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

/Users/ariellubonja/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you 

In [5]:
encoded_inputs = tokenizer(unique_drug_names, padding=True, truncation=True, return_tensors='pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Macbook M1 Pro Runtimes

CPU - 1min 17 sec

MPS (torch 1.13) - Uses way too much memory (50GB+ - crashes) whereas CPU is max 13GB

MPS (torch 2.01) - 1min 15s - 18GB memory usage


<font color="red">Conclusion - not useful yet!</font>

In [7]:
with torch.no_grad():
    model_output = model(**encoded_inputs)
    embeddings = model_output.last_hidden_state

1min 15s ± 8.66 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [ ]:
embeddings.shape

torch.Size([8, 768])